#### Tarea 5
Hacer dos figuras GIF con datos de circulación atmosférica media de DJF, MAM, JJA, SON similar a las figuras de la Tarea 1.
Esto es:
Circulación zonal (u) y meridional (v) en alta (200 hPa) y baja (800 hPa) atmósfera para los meses de diciembre, enero y febrero en Sudamérica, utilizando datos de reanálisis ERA5.


In [2]:
import glob
import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da
import os
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.shapereader as shpreader
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt
import cartopy.mpl.geoaxes
from matplotlib import pyplot as plt, animation
from IPython.display import HTML, display
import imageio  # GIFS

### 1. operating Dataset.


In [3]:
# Opening the dataset
ERA_5 = xr.open_dataset(r'C:\Users\nicol\Desktop\Articulo\TAREAS\3. Visualizacion de datos\Archivos NC\ERA 5 GIFS.nc', chunks=-1)
ERA_5

c:\Users\nicol\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


<xarray.Dataset>
Dimensions:    (longitude: 261, latitude: 221, level: 3, time: 864)
Coordinates:
  * longitude  (longitude) float32 -90.0 -89.75 -89.5 ... -25.5 -25.25 -25.0
  * latitude   (latitude) float32 15.0 14.75 14.5 14.25 ... -39.5 -39.75 -40.0
  * level      (level) int32 200 500 800
  * time       (time) datetime64[ns] 1950-01-01 1950-02-01 ... 2021-12-01
Data variables:
    z          (time, level, latitude, longitude) float32 dask.array<chunksize=(864, 3, 221, 261), meta=np.ndarray>
    q          (time, level, latitude, longitude) float32 dask.array<chunksize=(864, 3, 221, 261), meta=np.ndarray>
    u          (time, level, latitude, longitude) float32 dask.array<chunksize=(864, 3, 221, 261), meta=np.ndarray>
    v          (time, level, latitude, longitude) float32 dask.array<chunksize=(864, 3, 221, 261), meta=np.ndarray>
    w          (time, level, latitude, longitude) float32 dask.array<chunksize=(864, 3, 221, 261), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-07-06 20:14:16 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [4]:
ERA_5_1951_2021 = ERA_5.sel(time=slice('1951-04-01T00:00:00.000000000', '2021-11-01T00:00:00.000000000'))
ERA_5_1951_2021  

<xarray.Dataset>
Dimensions:    (longitude: 261, latitude: 221, level: 3, time: 848)
Coordinates:
  * longitude  (longitude) float32 -90.0 -89.75 -89.5 ... -25.5 -25.25 -25.0
  * latitude   (latitude) float32 15.0 14.75 14.5 14.25 ... -39.5 -39.75 -40.0
  * level      (level) int32 200 500 800
  * time       (time) datetime64[ns] 1951-04-01 1951-05-01 ... 2021-11-01
Data variables:
    z          (time, level, latitude, longitude) float32 dask.array<chunksize=(848, 3, 221, 261), meta=np.ndarray>
    q          (time, level, latitude, longitude) float32 dask.array<chunksize=(848, 3, 221, 261), meta=np.ndarray>
    u          (time, level, latitude, longitude) float32 dask.array<chunksize=(848, 3, 221, 261), meta=np.ndarray>
    v          (time, level, latitude, longitude) float32 dask.array<chunksize=(848, 3, 221, 261), meta=np.ndarray>
    w          (time, level, latitude, longitude) float32 dask.array<chunksize=(848, 3, 221, 261), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2023-07-06 20:14:16 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [36]:
# Variables to be used for visualization
x, y = ERA_5_1951_2021['longitude'], ERA_5_1951_2021['latitude']
REGION = [-90, -25, -40, 15]
TITULOS = ['DJF', 'MAM', 'JJA', 'SON']

In [37]:
# Resampling data to QS.
Q_1951_2021 = ERA_5_1951_2021['q'].resample(time='QS-DEC').mean(dim="time") * 1000
U_1951_2021 = ERA_5_1951_2021['u'].resample(time='QS-DEC').mean(dim="time")
V_1951_2021 = ERA_5_1951_2021['v'].resample(time='QS-DEC').mean(dim="time")
W_1951_2021 = ERA_5_1951_2021['w'].resample(time='QS-DEC').mean(dim="time") * 100

#### Obtaining variables by their respective QS

In [6]:
Q_1951_2021_DJF = Q_1951_2021[3::4].mean('time')
Q_1951_2021_MAM = Q_1951_2021[0::4].mean('time')
Q_1951_2021_JJA = Q_1951_2021[1::4].mean('time')
Q_1951_2021_SON = Q_1951_2021[2::4].mean('time')

In [7]:
U_1951_2021_DJF = U_1951_2021[3::4].mean('time')
U_1951_2021_MAM = U_1951_2021[0::4].mean('time')
U_1951_2021_JJA = U_1951_2021[1::4].mean('time')
U_1951_2021_SON = U_1951_2021[2::4].mean('time')

In [8]:
V_1951_2021_DJF = V_1951_2021[3::4].mean('time')
V_1951_2021_MAM = V_1951_2021[0::4].mean('time')
V_1951_2021_JJA = V_1951_2021[1::4].mean('time')
V_1951_2021_SON = V_1951_2021[2::4].mean('time')

In [9]:
W_1951_2021_DJF = W_1951_2021[3::4].mean('time')
W_1951_2021_MAM = W_1951_2021[0::4].mean('time')
W_1951_2021_JJA = W_1951_2021[1::4].mean('time')
W_1951_2021_SON = W_1951_2021[2::4].mean('time')

In [10]:
Q_1951_2021_QS = [Q_1951_2021_DJF, Q_1951_2021_MAM,
               Q_1951_2021_JJA, Q_1951_2021_SON]
U_1951_2021_QS = [U_1951_2021_DJF, U_1951_2021_MAM,
               U_1951_2021_JJA, U_1951_2021_SON]
V_1951_2021_QS = [V_1951_2021_DJF, V_1951_2021_MAM,
               V_1951_2021_JJA, V_1951_2021_SON]
W_1951_2021_QS = [W_1951_2021_DJF, W_1951_2021_MAM,
               W_1951_2021_JJA, W_1951_2021_SON]
W_1951_2021_JJA.coords

Coordinates:
  * longitude  (longitude) float32 -90.0 -89.75 -89.5 ... -25.5 -25.25 -25.0
  * latitude   (latitude) float32 15.0 14.75 14.5 14.25 ... -39.5 -39.75 -40.0
  * level      (level) int32 200 500 800

### 2. Vientos verticales

In [31]:
for i in range(len(W_1951_2021_QS)):
    fig, axs = plt.subplots(
        subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(14, 6))

    # Plot w_mean at 500 hPa as an image
    w_500 = W_1951_2021_QS[i][1,:,:].plot.contourf(
        add_colorbar=False,  # Colorbar
        robust=True,  # Use 98th and 2nd percentiles for color range
        cmap="RdBu_r",  # Color map
        levels=np.arange(-10, 11, 2),  # Levels for coloring the map
        rasterized=True,  # Separate objects
        alpha=1,  # Transparency
        # interpolation='none'  # No interpolation (pixelated)
    )

    # Add streamlines of u and v variables at 200 hPa
    axs.streamplot(x, y, U_1951_2021_QS[i][0,:,:], V_1951_2021_QS[i][0,:,:],
                linewidth=0.5,  # Tail size of arrows
                arrowsize=0.5,  # Head size of arrows
                density=2,  # Arrow density
                color='k',  # Color
                transform=ccrs.PlateCarree())

    axs.coastlines(color='k', linewidth=2)  # Add coastlines
    axs.add_feature(cfeature.BORDERS, linestyle="-", color='k',
                    alpha=0.8, linewidth=2)  # Add borders

    axs.gridlines(draw_labels=["left", "bottom"],  # Position of grid labels
                linewidth=1,  # Gridline size
                color='k',  # Gridline color
                alpha=0.8,  # Transparency
                linestyle=':')  # Gridline style

    axs.set_extent(REGION, crs=ccrs.PlateCarree())  # Set the displayed region

    cb = fig.colorbar(w_500, ax=axs, extend="both",
                    orientation="vertical", label="Pa s$^{-1}$")  # Colorbar settings
    cb.set_ticks((np.arange(-8, 9, 4)))  # Set colorbar ticks
    cb.minorticks_off()  # Disable minor ticks on colorbar

    # Set the plot title
    plt.title(f'W 500 hPa U-V 200 hPa, QS {TITULOS[i]}')
    plt.savefig(f"./Imagenes/Circulacion_200-QS-{TITULOS[i]}.png", bbox_inches='tight')  # Save the plot as a PNG file
    plt.close()

c:\Users\nicol\anaconda3\lib\site-packages\cartopy\mpl\geoaxes.py:1662: UserWarning: The following kwargs were not used by contour: 'rasterized'
  result = super().contourf(*args, **kwargs)
c:\Users\nicol\anaconda3\lib\site-packages\cartopy\mpl\style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '
c:\Users\nicol\anaconda3\lib\site-packages\cartopy\mpl\geoaxes.py:1662: UserWarning: The following kwargs were not used by contour: 'rasterized'
  result = super().contourf(*args, **kwargs)
c:\Users\nicol\anaconda3\lib\site-packages\cartopy\mpl\style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '
c:\Users\nicol\anaconda3\lib\site-packages\cartopy\mpl\geoaxes.py:1662: UserWarning: The following kwargs were not used by contour: 'rasterized'
  result = super().contourf(*args, **kwargs)
c:\Users\n

### 3. Humedad especifica

In [32]:
for i in range(len(Q_1951_2021_QS)):
    fig, axs = plt.subplots(
        subplot_kw={"projection": ccrs.PlateCarree()}, figsize=(14, 6))

    # Plot vertical wind levels
    q_plot = Q_1951_2021_QS[i][1, :, :].plot(
        add_colorbar=False,  # Add colorbar
        robust=True,  # Use 98th and 2nd percentiles for color range
        cmap="viridis_r",  # Color map
        levels=12,  # Levels for coloring
        rasterized=True,
        alpha=1
    )

    # Streamplot for arrows
    axs.streamplot(x, y, U_1951_2021_QS[i][2,:,:], V_1951_2021_QS[i][2,:,:],
                linewidth=0.5,  # Line width
                arrowsize=0.5,  # Arrow size
                density=2,  # Density of lines
                color='k',  # Color
                transform=ccrs.PlateCarree()  # Coordinate transformation
                )

    # Add coastlines and borders
    axs.coastlines(linewidth=2, color='white')
    axs.add_feature(cfeature.BORDERS, linestyle="-",
                    color='white', alpha=0.8, linewidth=2)

    # Add gridlines
    axs.gridlines(draw_labels=["left", "bottom"],
                linewidth=1,
                color='k',
                alpha=0.8,
                linestyle=':')

    axs.set_extent(REGION, crs=ccrs.PlateCarree())  # Set the displayed region


    # Add colorbar
    cb = fig.colorbar(q_plot,
                    ax=axs,
                    pad=0.09,
                    extend="neither",
                    orientation="vertical",
                    label='g kg$^{-1}$',
                    ticks=np.arange(0,5,0.4)
                    )

    # Set colorbar label size
    cb.ax.tick_params(labelsize=16)
    q_plot.set_alpha(0.8)
    plt.title('q at 500 hPa (shaded);\n u-v at 800 hPa (vector)')

    # Save the file
    plt.title(f'Q 500 hPa U-V 800 hPa, QS {TITULOS[i]}')
    plt.savefig(f"./Imagenes/Circulacion_800-QS-{TITULOS[i]}.png", bbox_inches='tight')  # Save the plot as a PNG file
    plt.close()

c:\Users\nicol\anaconda3\lib\site-packages\cartopy\mpl\style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '
c:\Users\nicol\anaconda3\lib\site-packages\cartopy\mpl\style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '
c:\Users\nicol\anaconda3\lib\site-packages\cartopy\mpl\style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '
c:\Users\nicol\anaconda3\lib\site-packages\cartopy\mpl\style.py:76: UserWarning: facecolor will have no effect as it has been defined as "never".
  warnings.warn('facecolor will have no effect as it has been '


### 4. GIFS

In [33]:
 
# Leer todos los archivos .png en el directorio "Images"
import glob
circulacion200 = sorted(glob.glob(f'./Imagenes/Circulacion_200*.png'), key=len)
circulacion800 = sorted(glob.glob(f'./Imagenes/Circulacion_800*.png'), key=len)


In [34]:
from PIL import Image

# Lista que tendra las imagenes
imagenes_circulacion_200 = []
imagenes_circulacion_800 = []
for file in circulacion200:
    # Abre la imagen
    image = Image.open(file)
    # La agrega a la lista
    imagenes_circulacion_200.append(image)

for file in circulacion800:
    # Abre la imagen
    image = Image.open(file)
    # La agrega a la lista
    imagenes_circulacion_800.append(image)

In [35]:
import imageio

# Guarda las imágenes como un archivo GIF
circulacion200_gif = "./Imagenes/Circulacion_200.gif"
imageio.mimsave(circulacion200_gif, imagenes_circulacion_200, duration=2)
circulacion800_gif = "./Imagenes/Circulacion_800.gif"
imageio.mimsave(circulacion800_gif, imagenes_circulacion_800, duration=2)

# Imprime un mensaje de confirmación
print(f"Los GIFs se ha creado correctamente y se han guardado como {circulacion200_gif,circulacion800_gif}!")

Los GIFs se ha creado correctamente y se han guardado como ('./Imagenes/Circulacion_200.gif', './Imagenes/Circulacion_800.gif')!
